In [1]:
import numba as nb
import torch
import numpy as np 
import os
import sys
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import time
import torch.optim as optim
os.chdir('/home/baly/projects/sopt_shape')
from lib.library import *
from lib.lib_ot import *
from lib.sliced_opt import *
from lib.lib_shape import *

#import opt1d
# !python --version
# from google.colab import drive
# !pip install numpy --upgrade
# !pip install numba --upgrade

In [7]:
# load data 

label='male'
label_index='_2_49'
data=torch.load('data/%s%s.pt'%(label,label_index))
# data=torch.load('data/%s_%d_%d.pt'%(label,i,j))

method_list={
    'SOPT_GD':{'method_func':SOPT_GD,'n_iteration': 2000,'start_epoch':200},
    'SOPT_RBF':{'method_func':SOPT_RBF,'n_iteration': 200,'start_epoch':20},
    'SOPT_TPS':{'method_func':SOPT_TPS,'n_iteration': 200,'start_epoch':20},
    'OPT_RBF':{'method_func':OPT_RBF,'n_iteration': 200,'start_epoch':20},
    'OPT_TPS':{'method_func':OPT_TPS,'n_iteration': 200,'start_epoch':20}
}


In [ ]:



for eta in [0.05,0.10]:
    save_path='result/3D_%s_%0.2f/'%(label,eta)
    print(save_path)
    X0=data['X0']
    N0=X0.shape[0]
    X=data['X'+str(eta)]
    Y=data['Y'+str(eta)]
    sigma2,eps=0.1,1.0
    device='cuda'

    for method in method_list:
        print(method)
        method_func,n_iteration,start_epoch=method_list[method]['method_func'],method_list[method]['n_iteration'],method_list[method]['start_epoch']
        start_time=time.time()
        if 'SOPT' in method:
            N0+=20
        model_list,record_index=method_func(X,Y,N0,n_iteration=n_iteration,start_epoch=start_epoch,threshold=0.8,device=device,numItermax=100000, numThreads=10)

        end_time=time.time()
        torch.save((model_list,record_index,end_time-start_time),save_path+'%s.pt'%(method))
        print('end')

result/3D_male_0.05/
SOPT_GD
endrent, 1999/2000
SOPT_RBF


/home/baly/projects/sopt_shape/lib/lib_shape.py:390: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3571.)
  scaling=torch.sum(torch.abs(S.T))/torch.trace(Y_tc.T.mm(Y_tc))


endrent, 199/200
SOPT_TPS


In [12]:
def model_to_Yhat(model_list,X,model='RBF'):
  Yhat_list=list()
  if model=='RBF':
    R_list,beta_list,alpha_list,Phi=model_list
    n_iteration=len(alpha_list)
    for epoch, (R,beta,alpha) in enumerate(zip(R_list,beta_list,alpha_list)):
      Yhat=Phi.dot(alpha)+X.dot(R)+beta
      Yhat_list.append(Yhat)

  if model=='TPS':
    X_bar=np.hstack((np.ones((X.shape[0],1)),X))
    B_list,alpha_list,Phi=model_list
    n_iteration=len(alpha_list)
    for epoch,(B,alpha) in enumerate(zip(B_list,alpha_list)):
      Yhat=Phi.dot(alpha)+X_bar.dot(B)
      Yhat_list.append(Yhat)

  return Yhat_list



def make_plot(X,Y,file_name=None):
  fig = plt.figure(figsize=(2*800/72,800/72))
  ax = fig.add_subplot(projection='3d')
  x=X[:,0]
  y=X[:,1]
  z=X[:,2]
  ax.scatter3D(X[:,0], X[:,2], X[:,1], s=0.5,c='r',alpha=0.5) #=X[:,1]*np.sqrt(X[:,0]**2+X[:,2]**2), cmap='bone')

  ax.scatter3D(Y[:,0], Y[:,2], Y[:,1], s=0.5,c='b',alpha=0.5) #Y[:,1]*np.sqrt(Y[:,0]**2+Y[:,2]**2), cmap='bone')

  ax.set_xlim([-1,1]);ax.set_ylim([-1,1]);ax.set_zlim([-1,1])
  ax.view_init(10, 45)
  if file_name!=None:
      plt.savefig(file_name,dpi=200)
  plt.show()

def visual_3D(Yhat_list,Y,path,record_index):
  for (Yhat,index) in zip(Yhat_list,record_index):
    file_name=path+'_'+str(index)+'.pdf'
    make_plot(Yhat,Y,file_name)

    
for eta in [0.05,0.10]:
    X=data['X'+str(eta)]
    Y=data['Y'+str(eta)]
    save_path='result/3D_male_%0.2f/'%eta
    for method in method_list:
        model_list,record_index,runing_time=torch.load(save_path+'%s.pt'%(method))        
        print('method is %s, rocord_index is %s'%(method,record_index))
        if 'RBF' in method or 'GD' in method:
            model='RBF'
        elif 'TPS' in method:
            model='TPS'
        print('<<<<<<<<<<method %s'%method)
        # Yhat_list=model_to_Yhat(model_list,X,model=model)
        # path=save_path+'pic/'+method
        # visual_3D(Yhat_list,Y,path,record_index)
        print('<<<<<<<end<<<<<<<<<<<<')


method is SOPT_GD, rocord_index is [   0  105  210  315  420  526  631  736  841  946 1052 1157 1262 1367
 1472 1578 1683 1788 1893 1999]
<<<<<<<<<<method SOPT_GD
<<<<<<<end<<<<<<<<<<<<
method is SOPT_RBF, rocord_index is [  0  10  20  31  41  52  62  73  83  94 104 115 125 136 146 157 167 178
 188 199]
<<<<<<<<<<method SOPT_RBF
<<<<<<<end<<<<<<<<<<<<
method is SOPT_TPS, rocord_index is [  0  10  20  31  41  52  62  73  83  94 104 115 125 136 146 157 167 178
 188 199]
<<<<<<<<<<method SOPT_TPS
<<<<<<<end<<<<<<<<<<<<
method is OPT_RBF, rocord_index is [  0  10  20  31  41  52  62  73  83  94 104 115 125 136 146 157 167 178
 188 199]
<<<<<<<<<<method OPT_RBF
<<<<<<<end<<<<<<<<<<<<
method is OPT_TPS, rocord_index is [  0  10  20  31  41  52  62  73  83  94 104 115 125 136 146 157 167 178
 188 199]
<<<<<<<<<<method OPT_TPS
<<<<<<<end<<<<<<<<<<<<
method is SOPT_GD, rocord_index is [   0  105  210  315  420  526  631  736  841  946 1052 1157 1262 1367
 1472 1578 1683 1788 1893 1999]
<<<<<<<<

In [14]:
method='OPT_TPS'
model_list,record_index,runing_time=torch.load(save_path+'%s.pt'%(method))  
print(len(model_list[0]))
print(len(record_index))

20
20


In [9]:
print('test')

test


In [ ]:
# test model
i=19
j=2

eta=0.05
label='female'

data=torch.load('data/%s_%d_%d.pt'%(label,i,j))
print(label)
record_index=[0,1,5,10]



X0=data['X0']
N0=X0.shape[0]
X=data['X0']
Y=data['Y0']


sigma2=0.1
eps=1

sample_index=np.unique(np.linspace(0,N0-1,800).astype(np.int64))
C=X.copy() #[sample_index] #.copy() #[0:N0] #[sample_index] #[0:N0].copy() 
Phi=kernel_matrix_Gaussian(C,X[0:N0],sigma2) 
K,D=C.shape
Yhat_op=Y[correspondence]

Yhat=Yhat_op.copy()
alpha=np.zeros((C.shape))
for i in range(1):
    Y2=Yhat-Phi.dot(alpha)
    R,S=recover_rotation(Y2,X)
    #make_plot(X.dot(R)+beta,Y)
    beta=vec_mean(Y2)-vec_mean(X.dot(R))
    Y=Yhat-X.dot(R)-beta
    alpha=recover_alpha(Phi,Y,epsilon=0.1)
    Yhat=Phi.dot(alpha)+X.dot(R)+beta
    #make_plot(Yhat,Y)
print('done')
make_plot(Yhat,Y)
print('----')

Yhat=Yhat_op.copy()
Phi=kernel_matrix_TPS(C,X,D=2)
X_bar=np.hstack((np.ones((X.shape[0],1)),X))
alpha,B=TPS_recover_parameter_cuda(Phi,X_bar,Yhat,epsilon=0.5)

Yhat2=Phi.dot(alpha)+X_bar.dot(B) #Phi.dot(alpha)+X.dot(R)+beta     
make_plot(Yhat2,Y)


In [ ]:
# do not run this one 
label='neutral'
data_list=torch.load('data/'+label+'_list.pt')
#data_list=np.array(data_list)
data_all={}

X=data_list[0] 
Y=data_list[-1]
n,d=X.shape


fig = plt.figure(figsize=(2*800/72,800/72))    
for n,i in enumerate(np.linspace(-.5,.5,5)):
    ax = fig.add_subplot(1, 5, n+1, projection='3d')
    data=data_list[n]
    x=data[:,0]
    y=data[:,1]
    z=data[:,2]
    ax.scatter3D(x, z, y, c=y*np.sqrt(x**2+z**2), cmap='bone')
    ax.set_xlim([-1,1]);ax.set_ylim([-1,1]);ax.set_zlim([-1,1])
    ax.view_init(10, 45)
plt.show()
data_all['0']=data_list


# generate 5% noisy data
n,d=X.shape
data_listY1=[]
eta=0.05
n_noise=int(n*eta)
fig = plt.figure(figsize=(2*800/72,800/72))   
for n,data in enumerate(data_list):

    noise=np.random.uniform(low=-1.5,high=1.5,size=(n_noise,d))
    data=np.concatenate((data,noise))
    x=data[:,0]
    y=data[:,1]
    z=data[:,2]
    if n<=4:
        ax = fig.add_subplot(1, 5, n+1, projection='3d')
        ax.scatter3D(x, z, y, c=y*np.sqrt(x**2+z**2), cmap='bone')
        ax.set_xlim([-1,1]);ax.set_ylim([-1,1]);ax.set_zlim([-1,1])
        ax.view_init(10, 45)
    data_listY1.append(data)
plt.show()

data_all[str(eta)]=data_listY1


# generate 10% noisy data
n,d=X.shape
data_listY1=[]
eta=0.10
n_noise=int(n*eta)
fig = plt.figure(figsize=(2*800/72,800/72))   
for n,data in enumerate(data_list):
    
    noise=np.random.uniform(low=-1.5,high=1.5,size=(n_noise,d))
    data=np.concatenate((data,noise))
    if n<=4:
        ax = fig.add_subplot(1, 5, n+1, projection='3d')
        x=data[:,0]
        y=data[:,1]
        z=data[:,2]
        ax.scatter3D(x, z, y, c=y*np.sqrt(x**2+z**2), cmap='bone')
        ax.set_xlim([-1,1]);ax.set_ylim([-1,1]);ax.set_zlim([-1,1])
        ax.view_init(10, 45)
    data_listY1.append(data)
plt.show()
data_all[str(eta)]=data_listY1

torch.save(data_all,'data/select_'+label+'.pt')


In [ ]:

@nb.njit()
def permutation_inverse(permutation):
    N=permutation.shape[0]
    Domain=np.arange(N)
    mapping=np.stack((Domain,permutation))
    mapping_final=mapping[0].take(mapping[1].argsort())
    return mapping_final

In [ ]:
# do not run this one 
label='male'
data_all=torch.load('data/select_%s.pt'%label)
i=2
j=49

X0=data_all['0'][i]
Y0=data_all['0'][j]
X1=data_all['0'][i]
Y1=data_all['0'][j]
make_plot(X1,Y1)
data={}
for eta in [0, 0.05,0.10]:
    X=data_all[str(eta)][i]
    Y=data_all[str(eta)][j]
    theta_0=np.array([np.pi/6,-np.pi/5,np.pi/6])
    rotation_0=rotation_in(theta_0)
    beta_0=np.array([-0.6,-0.3,0.1])
    S_0=1.0 #0.8
    permutation=np.random.permutation(N0)
    X3=X[0:N0][permutation]
    X1=X3.dot(rotation_0)+beta_0
    X1=np.concatenate((X1,X[N0:]))
    
    theta_op=-theta_0 #np.array([np.pi/6,np.pi/4,-np.pi/5])
    rotation_op=rotation_re(theta_op)
    S_op=1/S_0
    beta_op=-S_op*beta_0.dot(rotation_op)
    X2=X1.dot(rotation_op)*S_op+beta_op
    print(np.linalg.norm(X2[0:N0]-X[0:N0][permutation]))

    data['X'+str(eta)]=X1 
    data['Y'+str(eta)]=Y
    data['permutation'+str(eta)]=permutation
    data['theta_op']=theta_op
    data['rotation_op']=rotation_op
    data['S_op']=S_op
torch.save(data,'data/%s_%d_%d.pt'%(label,i,j))

make_plot(X1,Y1)
